In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

import warnings
warnings.filterwarnings("ignore")

np.random.seed(42)

def generate_hierarchical_data(n_periods=365):
    time = np.arange(n_periods)

    seasonal = 10 * np.sin(2 * np.pi * time / 7)
    trend = 0.05 * time
    noise = np.random.normal(0, 2, n_periods)

    region_weights = [0.25, 0.30, 0.20, 0.25]
    regions = {}

    for i, w in enumerate(region_weights):
        regions[f"Region_{i+1}"] = (
            w * (50 + trend + seasonal) +
            np.random.normal(0, 1.5, n_periods)
        )

    national = sum(regions.values()) + noise

    df = pd.DataFrame(regions)
    df["National"] = national
    return df

data = generate_hierarchical_data()
data.head()

train_size = int(len(data) * 0.8)
train = data.iloc[:train_size]
test = data.iloc[train_size:]

plot_acf(train["National"], lags=30)
plt.show()

plot_pacf(train["National"], lags=30)
plt.show()

model = SARIMAX(
    train["National"],
    order=(1,1,1),
    seasonal_order=(1,1,1,7),
    enforce_stationarity=False,
    enforce_invertibility=False
)

result = model.fit()
print(result.summary())

residuals = result.resid

plt.figure(figsize=(10,4))
plt.plot(residuals)
plt.title("Residuals")
plt.show()

plot_acf(residuals, lags=30)
plt.show()

def rolling_forecast(series, order, seasonal_order):
    predictions = []
    history = list(series[:train_size])

    for t in range(len(series) - train_size):
        model = SARIMAX(
            history,
            order=order,
            seasonal_order=seasonal_order,
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        result = model.fit(disp=False)
        yhat = result.forecast()[0]
        predictions.append(yhat)
        history.append(series[train_size + t])

    return np.array(predictions)

national_preds = rolling_forecast(
    data["National"].values,
    order=(1,1,1),
    seasonal_order=(1,1,1,7)
)

region_forecasts = {}

for col in data.columns[:-1]:
    model = SARIMAX(
        train[col],
        order=(1,1,1),
        seasonal_order=(1,1,1,7),
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    result = model.fit()
    region_forecasts[col] = result.forecast(len(test))

bottom_up_forecast = sum(region_forecasts.values())

rmse_ind = np.sqrt(mean_squared_error(test["National"], national_preds))
mape_ind = mean_absolute_percentage_error(test["National"], national_preds)

rmse_rec = np.sqrt(mean_squared_error(test["National"], bottom_up_forecast))
mape_rec = mean_absolute_percentage_error(test["National"], bottom_up_forecast)

print("Independent Forecast RMSE:", rmse_ind)
print("Independent Forecast MAPE:", mape_ind)

print("Bottom-Up Forecast RMSE:", rmse_rec)
print("Bottom-Up Forecast MAPE:", mape_rec)

final_forecasts = {}

for col in data.columns:
    model = SARIMAX(
        data[col],
        order=(1,1,1),
        seasonal_order=(1,1,1,7),
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    result = model.fit()
    final_forecasts[col] = result.forecast(30)

final_forecast_df = pd.DataFrame(final_forecasts)
final_forecast_df

final_forecast_df.to_csv("final_30_step_forecast.csv", index=False)
